In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn import svm
from sklearn import metrics
import scipy.io
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats, linalg
import warnings
import math
from sklearn.exceptions import ConvergenceWarning
def conv_r2z(r):
    with np.errstate(invalid='ignore', divide='ignore'):
        return 0.5 * (np.log(1 + r) - np.log(1 - r))
def conv_z2r(z):
    with np.errstate(invalid='ignore', divide='ignore'):
        return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)
def get_info(name):
    tmp = name.split('_')
    sub = tmp[0]
    ses = tmp[1]
    return sub, ses

os.chdir('/gpfs/milgram/project/chun/jk2992/rest_thoughts/') # change to your folder path

In [2]:
# get subject list
file_list = os.listdir('./data/brain/netts')
file_list.sort()
print('We have ' + str(len(file_list)) + ' rest functional runs')

scan_info = []
for i in range(len(file_list)):
    name = file_list[i]
    sub, ses = get_info(name)
    scan_info.append([sub,ses])
scan_info = np.asarray(scan_info)
sub_list = np.unique(scan_info[:,0])
print('We have ' + str(len(sub_list)) + ' participants')

one_ses_sub = []
two_ses_sub = []
for j in range(len(sub_list)):
    if scan_info[:,0].tolist().count(sub_list[j]) != 4:
        one_ses_sub.append(sub_list[j])
    else:
        two_ses_sub.append(sub_list[j])
print('We have ' + str(len(one_ses_sub)) + ' particpants with one rest session.')
print('They are ' + str(one_ses_sub))

# 1001, 1002, 1020 only did one session, 1054 second session scanner issue, 1044 converting issue

We have 231 rest functional runs
We have 60 participants
We have 5 particpants with one rest session.
They are ['1001', '1002', '1020', '1044', '1054']


In [3]:
rest = scipy.io.loadmat('./data/brain/rest_ts.mat')['ts']
print('TS.shape = ' + str(rest.shape))

TS.shape = (268, 597, 231)


In [4]:
def get_r_omit_nan(tc1,tc2):
    # view motion censored TRs as missing data
    tc1[np.where(tc1 == 0)] = np.nan
    tc2[np.where(tc2 == 0)] = np.nan
    # get nan position
    nanidx1 = np.where(np.isnan(tc1))
    nanidx2 = np.where(np.isnan(tc2))
    if len(nanidx1[0]) == len(nanidx2[0]):
        nanidx = np.unique(nanidx1+nanidx2)
        # don't calculate FC if less than 12 TR available
        if len(nanidx) > 18:
            zval = np.nan
        else:
            # remove nan from both tc
            tc1 = np.delete(tc1,nanidx)
            tc2 = np.delete(tc2,nanidx)
            # run correlation
            rval = stats.pearsonr(tc1,tc2)[0]
            zval = conv_r2z(rval)
    else:
        zval = np.nan
    return zval

In [5]:
# define a function to calculate functional connectivity within the rest time period
# trial = 0-7
def get_FC(tc, trial):
    time_range = range(trial*75 + 2, trial*75 + 32) # shifted time course for 5 seconds to account for the hymodynamic lag (+2 because the first 3 TRs was cut during preprocess)
    tc_shorter = tc[:,time_range]
    FC_vec = []
    for i in range(tc_shorter.shape[0]-1):
        for j in range(i+1,tc_shorter.shape[0]):
            # print('node1 ' +str(i))
            #print('node2 ' +str(j))
            tmp1 = tc_shorter[i,:]
            tmp2 = tc_shorter[j,:]
            rval = get_r_omit_nan(tmp1,tmp2)
            FC_vec.append(rval)
    FC_vec = np.asarray(FC_vec)
    return FC_vec

In [ ]:
# get brain data by subject
ntrial = 8
FC_all = []
for k in range(len(sub_list)):
    idx = [i for i in range(len(scan_info[:,0])) if scan_info[:,0][i] == sub_list[k]]
    FC_sub = []
    for run in range(len(idx)):
        print(str(idx[run]))
        FC_run = []
        tc = rest[:,:,idx[run]]
        for trial in range(ntrial):
            # print('ntrial '+str(trial))
            FC_trial = get_FC(tc,trial)
            FC_run.append(FC_trial)
        FC_run = np.asarray(FC_run)
        FC_sub.append(FC_run)
    FC_all.append(FC_sub)
scipy.io.savemat('./data/brain/rest_fc.mat',{'rest':FC_all})

0
